# Met-Encefalina

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
import molsysmt as molsysmt
import openmmtools as mmtools
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
from tqdm import tqdm

In [2]:
plt.style.use('ggplot')

## Definición del sistema

In [3]:
system_pdbfixer=PDBFixer(pdbid="1PLX")

In [4]:
for chain in system_pdbfixer.topology.chains():
    print("Chain index {} with pdb id {}".format(chain.id,chain.index))

Chain index A with pdb id 0


In [5]:
system_pdbfixer.findMissingResidues()
system_pdbfixer.findNonstandardResidues()
system_pdbfixer.findMissingAtoms()
system_pdbfixer.addMissingAtoms()
system_pdbfixer.addMissingHydrogens(pH=7.4)

In [11]:
view=molsysmt.view(system_pdbfixer)
view.clear()
view.add_ball_and_stick("protein")
view

NGLWidget()

In [9]:
aa=mm.System()

# Estado termodinámico

In [ ]:
# Formalismo NVT
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300*unit.kelvin
pressure    = None

# Parámetros de la simulación

In [ ]:
step_size       = 2*unit.femtoseconds
num_steps       = 400000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

# Integrador

In [ ]:
friction   = 1.0/unit.picosecond
integrator = mm.LangevinIntegrator(temperature, friction, step_size)

# Plataforma de simulación

In [ ]:
platform_name = 'CPU'  #platform:     A platform (CPU, OpenCL, CUDA, or reference); default is platform=OpenCL"
# for ii in range(mm.Platform.getNumPlatforms()):
#     print(mm.Platform.getPlatform(ii).getName())
platform = mm.Platform.getPlatformByName(platform_name)

# Reporteros y arrays de salida

In [ ]:
num_atoms  = system.topology.getNumAtoms()
times      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
positions  = unit.Quantity(np.zeros([num_steps_saved,num_atoms,3], np.float32), unit.angstroms)
velocities = unit.Quantity(np.zeros([num_steps_saved,num_atoms,3], np.float32), unit.angstroms/unit.picosecond)
potential_energies   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
kinetic_energies     = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)

# Condiciones iniciales

In [ ]:
initial_positions  = system.positions
#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico

context = mm.Context(system.system, integrator, platform)
context.setPositions(initial_positions)
context.setVelocitiesToTemperature(temperature)

In [ ]:
state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
times[0] = state.getTime()
positions[0] = state.getPositions()
velocities[0] = state.getVelocities()
kinetic_energies[0]=state.getKineticEnergy()
potential_energies[0]=state.getPotentialEnergy()

# Corriendo la simulación

In [ ]:
for ii in tqdm(range(num_steps_saved)):
    context.getIntegrator().step(saving_period)
    state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times[ii] = state.getTime()
    positions[ii] = state.getPositions()
    velocities[ii] = state.getVelocities()
    kinetic_energies[ii]=state.getKineticEnergy()
    potential_energies[ii]=state.getPotentialEnergy()

# Análisis de resultados

Accediendo a las posiciones:

In [ ]:
atom_index = 10 # Por ejemplo del átomo 10
plt.rcParams['figure.figsize'] = 18, 4
for ii, ylabel in zip(range(3),['X','Y','Z']):
    plt.plot(times,positions[:,atom_index,ii])
    plt.ylabel(ylabel+' ('+str(positions.unit)+')')
    plt.xlabel('time ('+str(times.unit)+')')
    plt.show()

Representando las energías cinética y potencial:

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
plt.plot(times,kinetic_energies[:])
plt.ylabel('Kinetic Energy ('+str(kinetic_energies.unit)+')')
plt.xlabel('time ('+str(times.unit)+')')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
plt.plot(times,potential_energies[:])
plt.ylabel('Potential Energy ('+str(potential_energies.unit)+')')
plt.xlabel('time ('+str(times.unit)+')')
plt.show()

Represento la trayectoria centrada y fitteada según los átomos de la cadena principal:

In [ ]:
mdtraj_topology = md.Topology.from_openmm(system.topology)
mdtrajectory    = md.Trajectory(positions/unit.nanometers, mdtraj_topology)

In [ ]:
mdtrajectory = mdtrajectory.center_coordinates()
mdtrajectory = mdtrajectory.superpose(reference=mdtrajectory, frame=0, 
                                      atom_indices=mdtrajectory.topology.select("backbone"))

In [ ]:
view_traj = nv.show_mdtraj(mdtrajectory)
view_traj

Represento el RMSD de la trayectoria fitteada respecto del frame 0 según los átomos de la cadena principal

In [ ]:
rmsd = md.rmsd(mdtrajectory, mdtrajectory, atom_indices=mdtrajectory.topology.select("backbone"))

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
plt.plot(times,rmsd[:])
plt.ylabel('RMSD (nm)')
plt.xlabel('time ('+str(times.unit)+')')
plt.show()

Represento los pares de ángulos dihedros phi y psi visitados por la trayectoria sobre el plot de ramachandran.

In [ ]:
phi   = md.compute_phi(mdtrajectory)
psi   = md.compute_psi(mdtrajectory)
omega = md.compute_omega(mdtrajectory)

In [ ]:
plt.rcParams['figure.figsize'] = 8, 8
plt.scatter(phi[1],psi[1])
plt.ylabel('PSI')
plt.xlabel('PHI')
plt.ylim(-np.pi,np.pi)
plt.xlim(-np.pi,np.pi)
plt.show()